In [8]:
import numpy as np
import random
from collections import defaultdict
from tqdm import tqdm
import pickle
# allow to import modules from the project root directory
import sys
import os

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from src.ParObsSnakeEnv import ParObsSnakeEnv
from src.FullObsSnakeEnv import FullObsSnakeEnv
from src.utils import compute_metrics
import matplotlib.pyplot as plt
import json

<img src="../../artifacts/images/SARSA.png" alt="SARSA algorithm" width="500" />

In [9]:
class SarsaAgent:
    def __init__(self, env, learning_rate=0.5, discount_factor=0.99, epsilon=0.1, learning_rate_decay=0.8, epsilon_decay=0.9):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.learning_rate_decay = learning_rate_decay
        self.epsilon_decay = epsilon_decay
        self.q_table = defaultdict(lambda: np.zeros(env.action_space.n))

    def choose_action(self, state):
        state = tuple(state.flatten())
        if random.uniform(0, 1) < self.epsilon:
            return self.env.action_space.sample(), None  # Explore
        else:
            return np.argmax(self.q_table[state]), None  # Exploit

    def update_q_value(self, state, action, reward, next_state, next_action):
        next_state = tuple(next_state.flatten())
        state = tuple(state.flatten())
        td_target = reward + self.discount_factor * self.q_table[next_state][next_action]
        td_error = td_target - self.q_table[state][action]
        self.q_table[state][action] += self.learning_rate * td_error
    
    def save_table(self, table_path):
        with open(table_path, 'wb') as f:
            pickle.dump(dict(self.q_table), f)
    
    def load_table(self, table_path):
        with open(table_path, 'rb') as f:
            self.q_table = pickle.load(f)

    def train(self, num_episodes, save_plots=False, plots_path='plots.png'):
        self.episode_rewards = []
        self.steps_per_episode = []
        self.epsilon_values = []
        self.learning_rate_values = []
        self.average_q_updates = []

        for episode in tqdm(range(num_episodes), desc='Training', unit='Episode'):
            state = self.env.reset()
            action, _ = self.choose_action(state)
            done = False
            total_reward = 0
            steps = 0
            q_update_magnitudes = []

            while not done:
                next_state, reward, done, _ = self.env.step(action)
                next_action, _ = self.choose_action(next_state)

                old_q_value = self.q_table[tuple(state.flatten())][action]
                self.update_q_value(state, action, reward, next_state, next_action)
                new_q_value = self.q_table[tuple(state.flatten())][action]
                q_update_magnitudes.append(abs(new_q_value - old_q_value))

                total_reward += reward
                state = next_state
                action = next_action
                steps += 1

            # Logging metrics for the episode
            self.episode_rewards.append(total_reward)
            self.steps_per_episode.append(steps)
            self.epsilon_values.append(self.epsilon)
            self.learning_rate_values.append(self.learning_rate)
            self.average_q_updates.append(np.mean(q_update_magnitudes))

            # Decay epsilon and learning rate periodically
            if episode % 1000 == 0 and episode != 0:
                self.epsilon *= self.epsilon_decay
                self.learning_rate *= self.learning_rate_decay

        # Save plots after training
        if save_plots:
            self.save_plots(plots_path)

    def save_plots(self, plots_path):
        plots_dir = os.path.dirname(plots_path)
        os.makedirs(plots_dir, exist_ok=True)

        # Create a figure with subplots
        fig, axs = plt.subplots(3, 2, figsize=(15, 15))

        # Plot rewards
        axs[0, 0].plot(self.episode_rewards)
        axs[0, 0].set_xlabel('Episode')
        axs[0, 0].set_ylabel('Cumulative Reward')
        axs[0, 0].set_title('Rewards Over Episodes')

        # Plot steps per episode
        axs[0, 1].plot(self.steps_per_episode)
        axs[0, 1].set_xlabel('Episode')
        axs[0, 1].set_ylabel('Steps per Episode')
        axs[0, 1].set_title('Steps Over Episodes')

        # Plot epsilon values
        axs[1, 0].plot(self.epsilon_values)
        axs[1, 0].set_xlabel('Episode')
        axs[1, 0].set_ylabel('Epsilon')
        axs[1, 0].set_title('Epsilon Decay Over Episodes')

        # Plot learning rate values
        axs[1, 1].plot(self.learning_rate_values)
        axs[1, 1].set_xlabel('Episode')
        axs[1, 1].set_ylabel('Learning Rate')
        axs[1, 1].set_title('Learning Rate Decay Over Episodes')

        # Plot Q-value updates
        axs[2, 0].plot(self.average_q_updates)
        axs[2, 0].set_xlabel('Episode')
        axs[2, 0].set_ylabel('Average Q-Value Update')
        axs[2, 0].set_title('Q-Value Updates Over Episodes')

        # Hide the empty subplot (bottom right)
        fig.delaxes(axs[2, 1])

        # Adjust layout
        plt.tight_layout()

        # Save the figure
        plt.savefig(plots_path)
        plt.close()

In [10]:
grid_size = 10
# env = FullObsSnakeEnv(grid_size=grid_size, interact=False)
env = ParObsSnakeEnv(grid_size=grid_size, interact=False)
agent = SarsaAgent(env, epsilon=0.1, discount_factor=0.9, learning_rate=0.9)

In [11]:
num_episodes = 50000
agent.train(num_episodes=num_episodes)

Training: 100%|██████████| 50000/50000 [01:28<00:00, 566.25Episode/s]


In [12]:
environment = 'full 'if isinstance(env, FullObsSnakeEnv) else 'par'

table_name = f'sarsa_table_{environment}_{num_episodes}_{grid_size}.pkl'
model_weights_dir = os.path.join('../..', 'models', 'sarsa')
os.makedirs(model_weights_dir, exist_ok=True)
table_path = os.path.join(model_weights_dir, table_name)

agent.save_table(table_path)

In [ ]:
if isinstance(env, ParObsSnakeEnv):
    env = ParObsSnakeEnv(grid_size=2*grid_size, interact=False)

model_metrics_dir = os.path.join('../..', 'artifacts', 'models_stats', 'sarsa')
os.makedirs(model_metrics_dir, exist_ok=True)

train_metrics_name = f'sarsa_train_metrics_{environment}_{num_episodes}_{grid_size}.png'
train_metrics_path = os.path.join(model_metrics_dir, train_metrics_name)
agent.save_plots(train_metrics_path)

num_simulations = 100
sim_metrics_name = f'sarsa_sim_metrics_{environment}_{num_episodes}_{env.grid_size}_{num_simulations}.json'
sim_metrics_path = os.path.join(model_metrics_dir, sim_metrics_name)
compute_metrics(agent, env, sim_metrics_path, num_simulations=num_simulations)

 62%|██████▏   | 62/100 [00:00<00:00, 303.89it/s]

Snake length: 20, Episode reward: 1550
Snake length: 19, Episode reward: 1531
Snake length: 23, Episode reward: 1848
Snake length: 37, Episode reward: 3170
Snake length: 26, Episode reward: 2176
Snake length: 19, Episode reward: 1496
Snake length: 14, Episode reward: 1039
Snake length: 25, Episode reward: 2102
Snake length: 36, Episode reward: 3033
Snake length: 31, Episode reward: 2564
Snake length: 24, Episode reward: 1950
Snake length: 49, Episode reward: 4297
Snake length: 22, Episode reward: 1750
Snake length: 32, Episode reward: 2672
Snake length: 7, Episode reward: 430
Snake length: 30, Episode reward: 2587
Snake length: 35, Episode reward: 2994
Snake length: 38, Episode reward: 3165
Snake length: 40, Episode reward: 3383
Snake length: 18, Episode reward: 1450
Snake length: 39, Episode reward: 3391
Snake length: 25, Episode reward: 2113
Snake length: 26, Episode reward: 2159
Snake length: 21, Episode reward: 1657
Snake length: 23, Episode reward: 1892
Snake length: 19, Episode r

100%|██████████| 100/100 [00:00<00:00, 279.22it/s]

Snake length: 36, Episode reward: 3092
Snake length: 24, Episode reward: 1981
Snake length: 5, Episode reward: 264
Snake length: 45, Episode reward: 3873
Snake length: 22, Episode reward: 1795
Snake length: 40, Episode reward: 3425
Snake length: 34, Episode reward: 2831
Snake length: 28, Episode reward: 2366
Snake length: 20, Episode reward: 1560
Snake length: 24, Episode reward: 1998
Snake length: 22, Episode reward: 1786
Snake length: 21, Episode reward: 1741
Snake length: 30, Episode reward: 2557
Snake length: 37, Episode reward: 3090
Snake length: 36, Episode reward: 3019
Snake length: 20, Episode reward: 1623
Snake length: 9, Episode reward: 676
Snake length: 42, Episode reward: 3541
Snake length: 39, Episode reward: 3273
Snake length: 24, Episode reward: 1980
Snake length: 31, Episode reward: 2644
Snake length: 49, Episode reward: 4189
Snake length: 26, Episode reward: 2116
Snake length: 19, Episode reward: 1552
Snake length: 25, Episode reward: 2012
Snake length: 44, Episode rew

{'snake_lengths': [20,
  19,
  23,
  37,
  26,
  19,
  14,
  25,
  36,
  31,
  24,
  49,
  22,
  32,
  7,
  30,
  35,
  38,
  40,
  18,
  39,
  25,
  26,
  21,
  23,
  19,
  29,
  26,
  22,
  20,
  19,
  16,
  30,
  24,
  9,
  30,
  25,
  26,
  27,
  22,
  29,
  38,
  36,
  19,
  8,
  17,
  17,
  30,
  17,
  32,
  26,
  41,
  29,
  27,
  12,
  28,
  25,
  8,
  19,
  19,
  32,
  32,
  45,
  31,
  36,
  24,
  5,
  45,
  22,
  40,
  34,
  28,
  20,
  24,
  22,
  21,
  30,
  37,
  36,
  20,
  9,
  42,
  39,
  24,
  31,
  49,
  26,
  19,
  25,
  44,
  21,
  29,
  28,
  18,
  34,
  28,
  14,
  32,
  30,
  29],
 'episode_rewards': [1550,
  1531,
  1848,
  3170,
  2176,
  1496,
  1039,
  2102,
  3033,
  2564,
  1950,
  4297,
  1750,
  2672,
  430,
  2587,
  2994,
  3165,
  3383,
  1450,
  3391,
  2113,
  2159,
  1657,
  1892,
  1507,
  2376,
  2098,
  1824,
  1636,
  1512,
  1260,
  2472,
  1954,
  658,
  2498,
  2032,
  2095,
  2234,
  1817,
  2425,
  3132,
  3070,
  1562,
  529,
  1353,
  13

In [14]:
if isinstance(env, FullObsSnakeEnv):
    env.interact = True
else:
    env = ParObsSnakeEnv(grid_size=2*grid_size)
    
state = env.reset()
done = False
while not done:
    action, _ = agent.choose_action(state)
    state, reward, done, _ = env.step(action)
    env.render()
    print(f"Reward: {reward}")

Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Rewa